In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [34]:
# Load the data
df = pd.read_csv('labeled-data.csv')

# Exclude examples labeled 'N' or 'Done'
df_filtered = df[(df['label'] != 'N') & (df['label'] != 'Done')]

# Extract features and labels
X = df_filtered.iloc[:, 2:9].values  # assuming your features start from the 3rd column
y = df_filtered['label'].values

In [35]:
# Specify the order of your classes
class_order = ['L', 'R']  # Adjust this based on your actual class order

# Initialize and fit the LabelEncoder with specified classes
label_encoder = LabelEncoder()
label_encoder.fit(class_order)

# Transform labels to encoded format
y_encoded = label_encoder.transform(y)

In [36]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Reshape the data for Conv1D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [40]:
# Build the CNN model with two convolutional layers
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
print("Shape after the first convolutional layer:", model.output_shape)

model.add(MaxPooling1D(pool_size=2))
print("Shape after the first convolutional layer:", model.layers[1].input_shape)
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so 1 neuron with sigmoid activation

Shape after the first convolutional layer: (None, 5, 32)
Shape after the first convolutional layer: (None, 5, 32)


ValueError: Exception encountered when calling layer "max_pooling1d_13" (type MaxPooling1D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling1d_13/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 1, 1], padding="VALID", strides=[1, 2, 1, 1]](max_pooling1d_13/ExpandDims)' with input shapes: [?,1,1,64].

Call arguments received by layer "max_pooling1d_13" (type MaxPooling1D):
  • inputs=tf.Tensor(shape=(None, 1, 64), dtype=float32)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
25/25 [==============================] - 0s 6ms/step - loss: 0.7494 - accuracy: 0.5195 - val_loss: 0.6090 - val_accuracy: 0.6517
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6336 - accuracy: 0.6620 - val_loss: 0.5944 - val_accuracy: 0.6966
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6156 - accuracy: 0.6898 - val_loss: 0.5839 - val_accuracy: 0.7079
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6025 - accuracy: 0.6885 - val_loss: 0.5797 - val_accuracy: 0.7079
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5947 - accuracy: 0.6898 - val_loss: 0.5715 - val_accuracy: 0.7079
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5862 - accuracy: 0.6974 - val_loss: 0.5716 - val_accuracy: 0.7079
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5807 - accuracy: 0.6936 - val_loss: 0.5663 - val_accuracy: 0.7079
Epoch 8/10
25/25 [==

In [ ]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 2ms/step
Test Accuracy: 0.6530612244897959
